<a href="https://colab.research.google.com/github/bioinfoUQAM/Identification-Tracking-Cows/blob/main/models/notebooks/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics torch torchvision opencv-python pillow numpy


In [ ]:
import os
import urllib.request
import torch
import cv2
import numpy as np
from PIL import Image
from ultralytics import YOLO
from torchvision import models, transforms
from IPython.display import display


In [ ]:
NUM_CLASSES = 29
TOP_K = 3

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


In [ ]:
os.makedirs("models", exist_ok=True)

YOLO_URL = "https://github.com/bioinfoUQAM/Identification-Tracking-Cows/releases/download/v1.0-models/best_yolov8_detection.pt"
ID_URL = "https://github.com/bioinfoUQAM/Identification-Tracking-Cows/releases/download/v1.0-models/best_model_efficientnet_b0.ckpt"

yolo_path = "models/best_yolov8_detection.pt"
id_path = "models/best_model_efficientnet_b0.ckpt"

if not os.path.exists(yolo_path):
    print("Downloading YOLOv8 detection model...")
    urllib.request.urlretrieve(YOLO_URL, yolo_path)

if not os.path.exists(id_path):
    print("Downloading EfficientNet identification model...")
    urllib.request.urlretrieve(ID_URL, id_path)

print("Models ready.")


In [ ]:
# Load detection model
yolo = YOLO(yolo_path)

# Load identification model
model = models.efficientnet_b0(weights=None)
model.classifier[1] = torch.nn.Linear(
    model.classifier[1].in_features,
    NUM_CLASSES
)

ckpt = torch.load(id_path, map_location=device)
state_dict = ckpt["state_dict"] if "state_dict" in ckpt else ckpt
state_dict = {k.replace("model.", ""): v for k, v in state_dict.items()}

model.load_state_dict(state_dict, strict=False)
model.eval().to(device)

print("Models loaded successfully.")


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]


In [ ]:
img_pil = Image.open(image_path).convert("RGB")
results = yolo(image_path)

boxes = []
for r in results:
    for box in r.boxes:
        if r.names[int(box.cls)] == "cow":
            boxes.append(box.xyxy[0].cpu().numpy())

if len(boxes) == 0:
    raise RuntimeError("No cow detected in the image.")


In [ ]:
x1, y1, x2, y2 = map(int, boxes[0])
crop = img_pil.crop((x1, y1, x2, y2))

input_tensor = preprocess(crop).unsqueeze(0).to(device)

with torch.no_grad():
    probs = torch.softmax(model(input_tensor), dim=1)[0]

top_probs, top_ids = torch.topk(probs, TOP_K)

print("Top-3 identity predictions:\n")
for rank, (pid, prob) in enumerate(zip(top_ids, top_probs), 1):
    print(f"{rank}. Cow ID {pid.item():02d} — confidence {prob.item():.3f}")

display(crop)
